# 데이터 집계와 그룹연산

데이터 집계에서 다루어질 내용
> 하나 이상의 키(key)를 이용해서 pandas객체를 여러 조각으로 나누는 방법  
> 합계, 평균, 표준편자, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법  
> 정규화, 선형 회귀, 동급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법  
> 피벗테이블과 교차 알람표를 구하는 방법  
> 변위치 분석과 다른 통계집단을 분석 수행하는 방법  

## GroupBy 메카닉

그룹 연산의 첫번째 단계에서는 Series, DataFrame 같은 pandas객체나 아니면 다른 객체에 들어 있는 데이터를 하나 이상의 키를 기준으로 분리
<img src ="https://media.vlpt.us/images/ssongplay/post/030290c1-825e-4ac9-8be5-b5f792f85a23/image.png" width=35%>

각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며 모두 같은 타입일 필요는 없다. 
- 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
- DataFrame의 컬럼 이름을 지칭하는 값
- 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series객체
- 축 색인 혹은 색인내의 개별 이름에 대해 실행되는 함수 

In [88]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(10,size=(5)),
                   'data2' : np.random.randint(10,size=(5))})
df

,key1,key2,data1,data2
0,a,one,1,1
1,a,two,1,7
2,b,one,1,3
3,b,two,1,3
4,a,one,2,0


In [2]:
grouped = df['data1'].groupby(df['key1'])  #data1을 key1에 대해 그룹화 하겠다!
grouped

데이터를 key1 으로 묶고 각 그룹에서 data1의 평균 구하기   
```groupBy```객체의 ```mean```메서드를 사용

In [3]:
grouped.mean() 

key1
a    4.0
b    8.5
Name: data1, dtype: float64

여러개의 배열을 리스트로 넘기면 다음과 같은 결과로 두개의 색인으로 묶이고, 계층적인 색인을 가지는 ```Series```를 얻을 수 있음 

In [4]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()  #대분류:key1, 소분류:key2로 데이터를 나눔!
means

key1  key2
a     one     4.5
      two     3.0
b     one     8.0
      two     9.0
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,4.5,3.0
b,8.0,9.0


In [6]:
df

,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [7]:
df['data1']

0    5
1    3
2    8
3    9
4    4
Name: data1, dtype: int32

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2004, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()
#위의 데이터1에 순서대로 붙여줌

California  2005    3.0
            2006    8.0
Ohio        2004    5.0
            2005    9.0
            2006    4.0
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()
#하나의 데이터에 대해서 ex. df['data1'].groupby 는 'key1'으로 사용이 불가능, 전체에 대해서는 ex. df.groupyby 'key1'으로 사용 가능
#df['data1'].groupby('key1').mean() 이거는 불가능!!!!!

,data1,data2
key1,,
a,4.0,3.333333
b,8.5,9.000000


In [10]:
df

,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [11]:

df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one     4.5    3.0
     two     3.0    4.0
b    one     8.0    9.0
     two     9.0    9.0

In [12]:
df.groupby(['key1', 'key2']).size()  #각 분류에 해당하는 원소가 몇 개인가

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹간 순회하기

```groupby``` 객체는 iteration을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환

In [13]:
df

,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [14]:
for name, group in df.groupby('key1'):
    print(name)  #key1(a, b)
    print(group)  #key1에 해당하는 그룹

a
  key1 key2  data1  data2
0    a  one      5      3
1    a  two      3      4
4    a  one      4      3
b
  key1 key2  data1  data2
2    b  one      8      9
3    b  two      9      9


In [15]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))  #key1, key2(a-b, one-two)
    print(group)    #해당에 속하는 그룹

('a', 'one')
  key1 key2  data1  data2
0    a  one      5      3
4    a  one      4      3
('a', 'two')
  key1 key2  data1  data2
1    a  two      3      4
('b', 'one')
  key1 key2  data1  data2
2    b  one      8      9
('b', 'two')
  key1 key2  data1  data2
3    b  two      9      9


원하는 데이터만 고르기 위해서 그룹별 데이터를 사전형으로 쉽게 바꾸어 사용 가능 

In [16]:
pieces = dict(list(df.groupby('key1')))
print(pieces)  #key1에 해당하는 a, b가 key, 나머지가 값으로 딕셔너리 만들어짐!
pieces['b']

{'a':   key1 key2  data1  data2
0    a  one      5      3
1    a  two      3      4
4    a  one      4      3, 'b':   key1 key2  data1  data2
2    b  one      8      9
3    b  two      9      9}


,key1,key2,data1,data2
2,b,one,8,9
3,b,two,9,9


```axis = 0``` 에 대해서 그룹을 만드는데 다른 축으로 그룹을 만드는 것도 가능 

아래의 예제는 df의 컬럼을 dtype에 따라 그룹으로 묶기 가능

In [17]:
df

,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [18]:
print(df.dtypes)
grouped = df.groupby(df.dtypes, axis=1)  #열에 대해 그룹을 만듦
#이전 형식은 k1, k2에 대해서 d1, d2이 그룹으로 만들어짐
#열에 대해 그룹을 만듦으로써 d1이 모두 같은 그룹으로 묶이고 k1이 모두 같은 그룹으로 묶임!
#자연스럽게 그룹화의 조건은 k1, k2가 아닌 다른 형식(데이터 타입)으로 바뀜

key1     object
key2     object
data1     int32
data2     int32
dtype: object


In [19]:
for dtype, group in grouped:
    print(dtype)
    print(group)

int32
   data1  data2
0      5      3
1      3      4
2      8      9
3      9      9
4      4      3
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 컬럼이나 컬럼의 일부만 선택하기

In [20]:
df

,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [21]:
df.groupby('key1')['data1']  #key1으로 분류하는데 data1만 가져온다
#df.groupby('key1')[['data2']]

In [22]:
df['data1'].groupby(df['key1'])  #data1만 key1로 분류한다
a= df[['data2']].groupby(df['key1'])  #data2에 대한 것을 key1으로 그룹화
for index, data in a:
    print(index)
    print(data)

a
   data2
0      3
1      4
4      3
b
   data2
2      9
3      9


아래의 예는 데이터에서 data2컬럼에 대해서만 평균을 구하고 결과를 ```DataFrame```으로 받고 싶다면 아래와 같이 작성

In [23]:
df['data2'].groupby([df['key1'], df['key2']]).mean()

key1  key2
a     one     3.0
      two     4.0
b     one     9.0
      two     9.0
Name: data2, dtype: float64

In [89]:
df.groupby(['key1', 'key2'])['data2'].mean()  #위에랑 같은 의미임!!
#[['data2']]로 쓰면 표로 그려지고 ['data2']로 쓰면 글자만 프린팅됨

key1  key2
a     one     0.5
      two     7.0
b     one     3.0
      two     3.0
Name: data2, dtype: float64

In [25]:
s_grouped = df.groupby(['key1', 'key2'])['data2']  #위에랑 같은 의미임!!
print(s_grouped)
s_grouped.mean()

key1  key2
a     one     3.0
      two     4.0
b     one     9.0
      two     9.0
Name: data2, dtype: float64

### 사전과 Series에서 그룹핑하기

각 컬럼을 나타낼 그룹 목록이 있고, 그룹별로 컬럼의 값을 모두 더한다고 할 경우

In [93]:
people = pd.DataFrame(np.random.randint(10, size=(5,5)),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people


,a,b,c,d,e
Joe,6,7,0,1,1
Steve,0,2,9,6,7
Wes,6,8,9,8,7
Jim,7,4,4,7,3
Travis,4,9,4,7,9


In [94]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values  #2열의 1행, 2행을 NaN으로
people
#행은 iloc이니까 3은 포함되지 않음! 2행만 해당

,a,b,c,d,e
Joe,6,7.0,0.0,1,1
Steve,0,2.0,9.0,6,7
Wes,6,NaN,NaN,8,7
Jim,7,4.0,4.0,7,3
Travis,4,9.0,4.0,7,9


In [95]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [96]:
by_column = people.groupby(mapping, axis=1)  #축별로 = a열은 red, b열은 red...
by_column.sum()  #각 그룹별로 더함(Joe-red = 7+1+8(a, b, e))

,blue,red
Joe,1.0,14.0
Steve,15.0,9.0
Wes,8.0,13.0
Jim,11.0,14.0
Travis,11.0,22.0


In [97]:
map_series = pd.Series(mapping)
map_series


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [98]:
people

,a,b,c,d,e
Joe,6,7.0,0.0,1,1
Steve,0,2.0,9.0,6,7
Wes,6,NaN,NaN,8,7
Jim,7,4.0,4.0,7,3
Travis,4,9.0,4.0,7,9


In [101]:
people.groupby(map_series, axis=1).count()  #Wes는 NaN이 2개니까 총 합이 3개임..

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 함수로 그룹핑하기

In [33]:
people

,a,b,c,d,e
Joe,9,2.0,9.0,2,6
Steve,8,0.0,1.0,7,8
Wes,4,NaN,NaN,0,1
Jim,3,8.0,5.0,4,9
Travis,3,2.0,4.0,1,4


위의 people 데이터는  DataFrame은 사람의 이름을 색인값으로 사용.   
만약 사람의 이름의 길이 별로 그룹을 묶고 싶다면 길이가 담긴 배열을 만들어 넘기는 대신 ```len``` 함수 사용 가능. 

In [34]:
people.groupby(len).sum()  #인덱스의 길이에 따라 그룹으로 묶어줌!

,a,b,c,d,e
3,16,10.0,14.0,6,16
5,8,0.0,1.0,7,8
6,3,2.0,4.0,1,4


내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 ```Series```와 함께 섞어 쓰더라도 전혀 문제가 되지 않음

In [35]:
people

,a,b,c,d,e
Joe,9,2.0,9.0,2,6
Steve,8,0.0,1.0,7,8
Wes,4,NaN,NaN,0,1
Jim,3,8.0,5.0,4,9
Travis,3,2.0,4.0,1,4


In [36]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()  #여러 값이 있으면 작은 값으로! NaN은 무시
#3-one: Joe, Wes
#3-two: Jim
#5-one: Steve
#6-two: Travis

a    b    c  d  e
3 one  4  2.0  9.0  0  1
  two  3  8.0  5.0  4  9
5 one  8  0.0  1.0  7  8
6 two  3  2.0  4.0  1  4

### 색인 단계로 그룹핑하기

계층적으로 색인된 데이터는 축 색인의 단계중 하나를 사용해서 편리하게 집계 가능

In [37]:
#MultiIndex => 열의 이름이 문자, 숫자 둘 다 생김! name은 문자의 이름, 숫자의 이름
#여러 개의 인덱스를 이중리스트로 구현!
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.753677 -1.356812 -0.302862 -0.928657  0.566196
1     -0.614118  0.089671  0.223618  0.152532 -0.706251
2     -0.746697 -0.380282 -0.022948 -0.799079  1.476894
3      1.119502  0.119332 -0.192014  0.987205  0.729343

```level``` 예약어를 사용해서 레벨 번호나 이름을 넘기면 가능

In [38]:
hier_df.groupby(level='city', axis=1).sum()  #city로 그룹화를 하겠다! 축으로 하겠다 = 모든 행들의 값을 가져옴
#축이 멀티인덱스일 때 level을 이용하여 어떤 축으로 그룹화할지 선택
#칼럼을 기준으로 그룹화를 하는 것이기 때문에 axis = 1을 써줘야 함!

city,JP,US
0,-0.362461,-0.905997
1,-0.553719,-0.300829
2,0.677815,-1.149927
3,1.716548,1.046819


## 데이터 집계

In [39]:
df


,key1,key2,data1,data2
0,a,one,5,3
1,a,two,3,4
2,b,one,8,9
3,b,two,9,9
4,a,one,4,3


In [40]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.5)  #아무것도 안적으면 디폴트는 0.5
#ex. a = 9, 6, 4의 데이터인 경우
#a의 경우 3개의 숫자 중 중간 숫자를 가져옴(같은 값이면 그 값, ex. 4, 2, 2 => 2) ex = 6
#b의 경우 2개의 숫자이기 때문에 그 숫자의 평균
#0.75이면? 중간값과 가장 큰 값의 평균을 가져옴! ex = (6+9)/2 = 0.75
#0.25이면? 중간값과 가장 작은 값의 평균을 가져옴! ex = (6+4)/2 = 5

key1
a    4.0
b    8.5
Name: data1, dtype: float64

자신만의 데이터 집계함수를 사용하려면 배열의 ```agg```메서드에 해당 함수를 넣으면 됨

In [41]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)
#원래 있던 그룹(key1)을 기준으로 max-min을 실행

#DF에 대해서는 apply 사용, Group에 대해서는 agg 사용

c:\users\svbea\appdata\local\programs\python\python39\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,data1,data2
key1,,
a,2,1
b,1,0


```describe```같은 메서드는 데이터를 집계하지 않는데도 잘 작동

In [42]:
grouped.describe()  #그룹의 통계를 보여줌

data1                                           data2                     \
     count mean       std  min   25%  50%   75%  max count      mean      std   
key1                                                                            
a      3.0  4.0  1.000000  3.0  3.50  4.0  4.50  5.0   3.0  3.333333  0.57735   
b      2.0  8.5  0.707107  8.0  8.25  8.5  8.75  9.0   2.0  9.000000  0.00000   

                               
      min  25%  50%  75%  max  
key1                           
a     3.0  3.0  3.0  3.5  4.0  
b     9.0  9.0  9.0  9.0  9.0

### 컬럼에 여러가지 함수 적응하기

In [43]:
tips = pd.read_csv('./tips.csv')  #'tips.csv' 이거랑 './tips.csv' 이거랑 뭐가 다른거야? 둘 다 되는데?
tips.tail()

,total_bill,tip,smoker,day,time,size
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2
243,18.78,3.00,No,Thur,Dinner,2


In [44]:
tips = pd.read_csv('./tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']  #하나의 열을 더 만듦(금액 대비 팁 비율 = 퍼센테이지)
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


컬럼에 따라 다른 함수를 사용해서 집계를 수행 하거나 열개의 함수를 한번에 적용하기 원한다면 쉽고 간단하게 사용가능

In [45]:
grouped = tips.groupby(['day', 'smoker'])  #대분류:요일, 소분류:흡연여부

In [46]:
grouped_pct = grouped['tip_pct']  #요일, 흡연여부로 그룹화한 데이터에서 팁 퍼센테이지를 가져옴
grouped_pct.agg('mean')  #해당 그룹에 대해(퍼센테이지) 평균값을 계산함
#agg: 데이터에 대해 어떤 계산을 실행할 것인지, 어떤 결과로 출력할 것인지

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 ```DataFrame```을 얻을 수 있음

In [47]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
#평균, 표준편차, 최대-최소를 출력(그 요일에 많이 준사람-적게 준사람)

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [48]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})  #팁에 대해 최대값, 인원은 합(어느 요일에 방문객이 많았는가)

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [49]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],  #팁 퍼센테이지에 대한 통계들
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## Apply: 일반적인 분리-적용-병합

#### 상위 5개의 tip_pct 값을 고르기  
특정 칼럼에서 가장 큰 값을 갖는 행을 선택하는 함수 필요

In [50]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]  #오름차순이니까 뒤에서 가져와야 큰 값임
#by = tip_pct: 퍼센테이지를 조건으로 정렬! 퍼센테이지를 오름차순으로 정렬하겠다
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


smoker그룹에 대해서 이 함수를 적용하면 다음과 같은 결과

In [51]:
tips.groupby('smoker').apply(top)  #흡연에 대해 그룹화하여 top 함수를 적용

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

```apply``` 메서드를 넘길 함수가 추가적인 인자를 받는 다면 이 함수 이름 뒤에 붙여서 넘겨주면 가능 

In [52]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')
#담배와 요일로 그룹화, 구매금액 제일 많은 사람 1명만 가져옴
#apply 함수를 사용할 때, 내가 만든 top 함수의 인자를 바꾸고싶다면 옆에 콤마로 구분하여 넘김

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [53]:
result = tips.groupby('smoker')['tip_pct'].describe()
result


,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [54]:
result.unstack('smoker')  #데이터를 쌓지 않음(옆으로 늘여뜨림)
#unstack: 해당 데이터프레임을 시계방향으로 90도 돌림
#stack: 해당 데이터프레임을 반시계방향으로 90도 돌림

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [55]:
#grouped는 day, smoker로 그룹화를 해둔 데이터
f = lambda x: x.describe()
grouped.apply(f)


total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### 그룹 색인 생략하기

In [56]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [57]:
tips.groupby('smoker', group_keys=False).apply(top)
#group_keys = False로 설정함으로써 앞의 그룹화한 인덱스는 빼줌(아래의 그래프와의 차이)
#어차피 No, Yes로 그룹화 됐잖아!! 뭐하러 또 앞에 인덱스까지 넣어!! 라는 의미

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [58]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### 변위치 분석과 버킷분석

pandas의 ```cut```과 ```qcut```메서드를 사용하여 선택한 크기만큼 혹은 표분 변위치에 따라 데이터를 나눌 수 있음  
cut을 이용해서 등간격 구간으로 나누기


In [59]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)  #cut: 구간별로 나누는 것
#data1을 4구간으로 나눴을 때, 해당 값들이 어느 구간에 위치하는지를 알려줌!(cut)
quartiles[:10]

0     (1.598, 3.081]
1    (-1.368, 0.115]
2    (-1.368, 0.115]
3     (0.115, 1.598]
4     (0.115, 1.598]
5     (0.115, 1.598]
6     (0.115, 1.598]
7    (-1.368, 0.115]
8     (0.115, 1.598]
9    (-1.368, 0.115]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.857, -1.368] < (-1.368, 0.115] < (0.115, 1.598] < (1.598, 3.081]]

In [60]:
frame.data1[:10]

0    1.819683
1   -0.842066
2   -0.512260
3    0.577537
4    0.630749
5    1.223299
6    0.280634
7   -1.128352
8    0.332026
9   -0.187020
Name: data1, dtype: float64

```cut```에서 반환된 categorical객체는 바로 ```groupby```로 넘기기 가능 

In [61]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)  #구간을 조건으로 data2를 그룹화함
grouped.apply(get_stats).unstack()  #언제 unstack을 써야 데이터가 깔끔하게 보이는지 모르겠네..
#구간으로 나눴던 quartiles의 정보를 보여줌

,min,max,count,mean
data1,,,,
"(-2.857, -1.368]",-2.133077,2.251119,72.0,-0.093398
"(-1.368, 0.115]",-2.633916,2.888343,472.0,0.000788
"(0.115, 1.598]",-2.903091,2.775172,394.0,-0.017294
"(1.598, 3.081]",-2.764857,2.453746,62.0,0.082783


표본 변위치에 기반하여 크기각 같은 버킷을 계산하기 위해서는 ```qcut```을 사용

In [62]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)  #개수를 똑같이 cut함
#labels = False를 하지 않으면 각 구간의 라벨이 구간으로 됨, False를 하면 0, 1, 2 ...로 설정 => df에 넣었을 때 인덱스 이름으로 설정
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.380314,2.251119,100.0,-0.070947
1,-2.207614,2.888343,100.0,0.123808
2,-1.954540,2.822910,100.0,0.029232
3,-2.101048,2.711760,100.0,-0.122746
4,-2.204621,2.833360,100.0,-0.043246
5,-2.633916,2.775172,100.0,-0.026661
6,-2.076962,2.630581,100.0,0.173789
7,-2.509185,2.411562,100.0,-0.246201
8,-2.903091,2.417586,100.0,0.100295


### @@@@ 정리 @@@@

cut: 같은 길이로 자르기

qcut: 같은 개수로 자르기

### Example: 그룹에 따른 값으로 결측치 채우기

누락된 데이터를 정리할때 어떤 경우에는 dropna를 사용해서 데이터를 살펴보고 걸러내기가능  
어떤 경우에는 누락된 값을 고정된 값이나 데이터로부터 도출된 어떤 값으로 채우고 싶을때에는 fillna메서드를 사용
누락된 값을 평균값으로 대체 


In [63]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
#s.dropna()  를 실행하면 NaN 데이터는 없어짐

0         NaN
1    0.127974
2         NaN
3    1.150771
4         NaN
5    0.356268
dtype: float64

In [64]:
s.fillna(s.mean())  #NaN값을 s의 평균으로 채움

0    0.545004
1    0.127974
2    0.545004
3    1.150771
4    0.545004
5    0.356268
dtype: float64

In [65]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4  #앞의 4개는 East, 뒤의 4개는 West의 값을 줌
data = pd.Series(np.random.randint(10,size=(8)), index=states)
data

Ohio          3
New York      2
Vermont       2
Florida       1
Oregon        9
Nevada        5
California    6
Idaho         8
dtype: int32

In [66]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data


Ohio          3.0
New York      2.0
Vermont       NaN
Florida       1.0
Oregon        9.0
Nevada        NaN
California    6.0
Idaho         NaN
dtype: float64

In [67]:
data.groupby(group_key).mean()  #group key를 기준으로 평균을 냄
#위에서 4개씩 지정한 key를 바탕으로 그룹화하고 평균을 냄
#데이터가 NaN인 인덱스들은 통계계산을 하지 않음!
#East 그룹에서 NaN이 1개 = 전체합/3

East    2.0
West    7.5
dtype: float64

In [68]:
fill_mean = lambda g: g.fillna(g.mean())
data


Ohio          3.0
New York      2.0
Vermont       NaN
Florida       1.0
Oregon        9.0
Nevada        NaN
California    6.0
Idaho         NaN
dtype: float64

In [69]:
data.groupby(group_key).apply(fill_mean)  #평균값으로 NaN을 채워줌
#Vermont는 East의 평균으로, Nevada와 Idaho는 West의 평균으로 채움

Ohio          3.0
New York      2.0
Vermont       2.0
Florida       1.0
Oregon        9.0
Nevada        7.5
California    6.0
Idaho         7.5
dtype: float64

In [70]:
fill_values = {'East': 0.5, 'West': -1}  #정해준 값으루 NaN 채워주기!
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          3.0
New York      2.0
Vermont       0.5
Florida       1.0
Oregon        9.0
Nevada       -1.0
California    6.0
Idaho        -1.0
dtype: float64

### Example: 그룹의 가중 평균과 상관관계

In [71]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randint(10, size=8),
                   'weights': np.random.randint(10,size=8)})
df

,category,data,weights
0,a,1,1
1,a,3,8
2,a,6,9
3,a,7,7
4,b,0,1
5,b,5,8
6,b,8,4
7,b,4,0


In [72]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
#average는 데이터에 가중치를 함께 연산하여 구한 평균임!!
#avg = sum(a * weights) / sum(weights)
grouped.apply(get_wavg)

category
a    5.120000
b    5.538462
dtype: float64

야후의 파이낸스에서 가져온 몇몇 주식과 s&p 500 지수(종목코드 SPX)의 종가 데이터를 살펴보자

In [73]:
close_px = pd.read_csv('./stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


퍼센트의 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관 관계를 알아보기

In [74]:
spx_corr = lambda x: x.corrwith(x['SPX'])  #내가 가져온 데이터와 SPX의 상관관계

```pct_change```함수를 이용해서 close_px의 페선트 변화율을 계산

In [75]:
rets = close_px.pct_change().dropna()  #NaN 데이터부분을 지워버림
rets
#pct_change(): 다음 행과의 차이(변화율)임
#첫 번째 행은 비교할 값이 없어 NaN => 그래서 dropna를 하는 것

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


datetime에서 연도 속성만 반환하는 한줄짜리 함수를 이용하여 연도별 퍼센트 변화율

In [76]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)  #연도로 그룹화
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


아래는 애플과 마이크로 소프트의 주가의 연간 상관관계

In [77]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))
#연도로 그룹화한 데이터에서 애플과 마이크로소프트의 상관관계를 구해줌(연도별)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 피벗테이블과 교차 일람표

피벗테이블은 앞에서 설명한 groupby 기능을 사용해서 측정 색인 활용한 재형성 연산가능


DataFrame에는 pivot_table 메서드가능


In [78]:
tips.pivot_table(index=['day', 'smoker'])
#요일, 흡연여부를 인덱스로 사용하여 피벗테이블 생성(그룹화와 비슷함! 대분류 = 요일, 소분류 = 흡연여부)

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [79]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')
#데이터는 사이즈와 퍼센테이지, 인덱스는 시간과 요일, 열은 흡연여부
#디폴트 값은 평균!! mean이 agg함수의 기본값임

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [80]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)
#margins = True이면 각 값의 총계를 보여줌(aggfunc에 의해)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [81]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)
#퍼센테이지 데이터를 열: 요일, 인덱스: 시간 + 흡연여부로 나눔
#agg 함수(len)를 통해 해당 데이터의 수 == 인원수를 출력!!
#agg 함수의 len 방법은 해당 그룹에 속하는 데이터 수를 세줌!! 따라서 이 데이터에서는 요일별 인원수가 되는 것!

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [82]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)
#agg함수는 평균 = 팁 퍼센테이지에 대한 평균
#fill_value를 통해 NaN인 데이터를 0으로 채워줌

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [83]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [84]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [85]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)
#crosstab: 인덱스-국적, 칼럼-무슨손잡이, All을 출력해줌(피벗테이블이랑 비슷해!!)
#crosstab을 통해 첫 변수는 인덱스,두 번째는 칼럼, margins를 선택했으니 총계가 나옴

#crosstab은 자동으로 해당 데이터의 수를 카운트해줌!

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [86]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)
#time과 day를 인덱스로, smoker를 칼럼으로 데이터

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244